# Creating the pooled dataset

This notebook combines publicly available data from two previous papers:

- Tornhill and Borg. <b>Code Red</b>: The Business Impact of Code Quality - A Quantitative Study of 39 Proprietary Production Codebases, In Proc. of the 5th International Conference on Techincal Debt, 2022.
- Borg, Tornhill, and Mones. <b>U Owns the Code</b> That Changes and How Marginal Owners Resolve Issues Slower in Low-Quality Source Code, In Proc. of the 27th International Conference on Evaluation and Assessment in Software Engineering, 2023.

In [1]:
%reset -f

In [2]:
# packages

import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import sklearn
from scipy import stats

## Processing the Code Red data

We import the complete Code Red dataset as preprocessed in the original study. We remove entries with invalid time information, then we rename and reorder columns.

In [51]:
raw_df_CodeRed = pd.read_csv('./data/code_health_ct_data_updated.csv', sep=';', decimal=",")
valid_df_CodeRed = raw_df_CodeRed[(raw_df_CodeRed['mean_time_for_issue'] > 0)]
valid_df_CodeRed.rename(columns={"code_health_now": "code_health", "mean_time_for_issue": "lead_time_minutes"}, inplace=True)
clean_df_CodeRed=valid_df_CodeRed[['code_health', 'lead_time_minutes', 'total_defects', 'language']]
print(clean_df_CodeRed.describe())

        code_health  lead_time_minutes  total_defects
count  25386.000000       2.538600e+04   25386.000000
mean       9.548462       8.573125e+03       0.341842
std        1.174215       2.539253e+04       1.356946
min        1.000000       6.000000e+01       0.000000
25%        9.630000       1.560000e+03       0.000000
50%       10.000000       3.360000e+03       0.000000
75%       10.000000       8.014500e+03       0.000000
max       10.000000       1.452780e+06      58.000000


## Processing the U Owns the Code data

We first repeat the same processing as presented in the Jupyter Notebook accompanying the U Owns the Code paper. This results in a clean dataset, following the same argumentation as in the original study.

In [56]:
raw_df_UOwns = pd.read_csv('./data/pm_knowledge_impact.csv', decimal=".")

# Remove invalid entries
valid_df_UOwns = raw_df_UOwns[(raw_df_UOwns['lead_time_minutes'] > 0) & (raw_df_UOwns['added_loc'] > 0)]

# remove extremely long lead times and huge chunks of code
clean_df_UOwns = valid_df_UOwns[(np.abs(stats.zscore(valid_df_UOwns['lead_time_minutes'])) < 3) & 
                    (np.abs(stats.zscore(valid_df_UOwns['added_loc'])) < 3)]

print(clean_df_UOwns.describe())

       contributing_months_relative_norm  lead_time_minutes  module_ownership  \
count                       98696.000000       99387.000000      99387.000000   
mean                            0.809825        6547.571010          0.486823   
std                             0.319798       10718.583371          0.383404   
min                             0.000000          60.000000          0.000000   
25%                             0.675676         900.000000          0.101906   
50%                             1.000000        3000.000000          0.416667   
75%                             1.000000        7380.000000          0.923877   
max                             1.000000       89160.000000          1.000000   

       contributing_months    added_loc   code_health  
count         99387.000000  99387.00000  99387.000000  
mean             19.442835     19.18427      9.512839  
std              20.563435     35.99349      1.076344  
min               0.000000      1.00000      1

For convenience, we store a separate data frame with only Jira bug fixes.

In [57]:
bug_df_UOwns=clean_df_UOwns[clean_df_UOwns['bug_fix']==True] 

print(bug_df_UOwns.describe())

       contributing_months_relative_norm  lead_time_minutes  module_ownership  \
count                       14274.000000       14281.000000      14281.000000   
mean                            0.877200        4602.148309          0.380215   
std                             0.267515        9080.707411          0.353758   
min                             0.000000          60.000000          0.000000   
25%                             1.000000         180.000000          0.061538   
50%                             1.000000        1680.000000          0.247525   
75%                             1.000000        4680.000000          0.695987   
max                             1.000000       88260.000000          1.000000   

       contributing_months     added_loc   code_health  
count         14281.000000  14281.000000  14281.000000  
mean             22.482879     15.152090      9.283857  
std              22.931215     31.135755      1.281988  
min               0.000000      1.000000  

In contrast to the Code Red dataset, the U Owns the Code data represents <i>touches</i>, i.e., individual changes to files. To enable pooling of the data, we need to aggregate the data per file. We use the following approach:

We group all touches to the same file ('module') and in store three values:
- The average Code Health at the time of the touches.
- The average time-in-development for the touches.
- The total number of touches to the file that were bug fixes.

In [64]:
reshaped_df_UOwns = pd.concat([clean_df_UOwns.groupby('module')[['code_health','lead_time_minutes']].mean().reset_index().set_index('module'),
                    bug_df_UOwns.groupby('module')['ticket_id'].count().reset_index().set_index('module')], axis=1)

# we collect language information
lang_df=clean_df_UOwns[['module','programming_language']]
lang_df=lang_df.drop_duplicates().set_index('module')
reshaped_df_UOwns=reshaped_df_UOwns.join(lang_df)
    
reshaped_df_UOwns.rename(columns={"ticket_id": "total_defects", "programming_language": "language"}, inplace=True)

# set total_defects to 0 for missing values
reshaped_df_UOwns['total_defects'] = reshaped_df_UOwns['total_defects'].fillna(0)

print(reshaped_df_UOwns.describe())

        code_health  lead_time_minutes  total_defects
count  20825.000000       20825.000000   20825.000000
mean       9.673315        6194.114873       0.685762
std        0.924667        9034.932586       2.716461
min        1.000000          60.000000       0.000000
25%        9.750000        1260.000000       0.000000
50%       10.000000        3360.000000       0.000000
75%       10.000000        7440.000000       0.000000
max       10.000000       89160.000000     110.000000


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


## Merge the two dataframes and save the results

In [65]:
increasing_returns_df=pd.concat([clean_df_CodeRed,reshaped_df_UOwns])
print(increasing_returns_df.describe())

        code_health  lead_time_minutes  total_defects
count  46211.000000       4.621100e+04   46211.000000
mean       9.604727       7.501023e+03       0.496830
std        1.070784       1.980885e+04       2.089526
min        1.000000       6.000000e+01       0.000000
25%        9.750000       1.380000e+03       0.000000
50%       10.000000       3.360000e+03       0.000000
75%       10.000000       7.680000e+03       0.000000
max       10.000000       1.452780e+06     110.000000


In [66]:
increasing_returns_df.to_csv('increasing_returns.csv',index=False)